# Vector Creation

## Usage

Update the constants below to work with your file structure

HYPERSPECRAL_RASTER_TIF_PATH this should be the path to a tif file cointaining the hyperspectral data to be converted to a vector containing all attributes.

NUMBER_OF_BANDS this is the number of bands in the dataset following pre-processing.

GEOMETRY_FILE this is the file containing the geometry of each point in the raster (can extract from GIS by converting the raster (tif file) to point data with one attribute. The attribute table can be copied to Excel and saved as a csv for use.

VECTOR_SHP_PATH this will be where the generated data is placed for as a shape file to be used in GIS and further analysis. 

VECTOR_CSV_PATH this will be where the generated data is placed for further anlaysis.

In [ ]:
HYPERSPECRAL_RASTER_TIF_PATH = 'preprocessed_data/hyperspectral_data_for_site.tif'
NUMBER_OF_BANDS = 358
GEOMETRY_FILE = 'preprocessed_data/geometry_file_for_site.csv'
VECTOR_SHP_PATH = 'vector/site_name.shp'
VECTOR_CSV_PATH = 'vector/site_name.csv'

In [ ]:
import pandas as pd
import geopandas
import rasterio as rio
data = rio.open(HYPERSPECRAL_RASTER_TIF_PATH)

print(data.transform)
lat_long_NW = data.transform * (0, 0)
print(lat_long_NW)
lat_long_SE = data.transform * (data.width, data.height)
print(lat_long_SE)
print(data.crs)

# read bands
array = data.read()

data_df = pd.DataFrame()

array_num = 0
band_num = 1
min_lat = lat_long_NW[0]
min_long = lat_long_SE[1]
max_lat = lat_long_SE[0]
max_long = lat_long_NW[1]

for num in range(NUMBER_OF_BANDS):
    data_df['band' +str(band_num)]=array[array_num].ravel()
    array_num += 1
    band_num += 1

data_df.head()

In [ ]:
data_df["POINTID"] = ""


index_num = 0
point_num = 1

for index, row in data_df.iterrows():
    data_df.iloc[index_num, NUMBER_OF_BANDS] = point_num
    point_num += 1
    index_num += 1
    
data_df.head()

In [ ]:
geometry_file = pd.read_csv(GEOMETRY_FILE)

data_gdf = pd.merge(data_df, geometry_file, on="POINTID")

data_gdf = data_gdf.rename(columns={"wkt_geom": "geometry"})

In [ ]:
from shapely.geometry import Point

data_gdf['geometry'] = geopandas.GeoSeries.from_wkt(data_gdf['geometry'])

gdf = geopandas.GeoDataFrame(data_gdf, geometry='geometry')

In [ ]:
gdf.to_file(VECTOR_SHP_PATH)
gdf.to_csv(VECTOR_CSV_PATH)